# Importing libraries and defining path and filenames

In this block of code, the path and name of the files is defined, and the libraries that are going to be used are imported.

In [1]:
import pandas as pd
import pyodbc as pdb
from datetime import datetime
import os

mainpath ='/home/ale/Downloads'
f1 = '2010-02.csv'
f2 = '2021-02.csv'

# Connection to database

Here, the connection to the database is created.

In [2]:
server = 'localhost' 
database = 'TestDB' 
username = 'ale' 
password = 'my-pa55w0rd' 
cnxn = pdb.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password,autocommit=True)
cursor = cnxn.cursor()

# Table creation

The table where the data will be stored is created in the following code.

If the table already exists, it's dropped, and the it's created.

In [3]:
sql = "DROP TABLE IF EXISTS Viajes"
cursor.execute(sql)
sql = "CREATE TABLE Viajes(ID INT PRIMARY KEY, ID_bici INT, fecha DATE, duracion TIME, est_salida INT, est_llegada INT)"
cursor.execute(sql)
print('The table hass been created succesfully!')

The table hass been created succesfully!


# First transformations

From the first file, a dataframe is created, giving the data the exact format for the table to receive it.

In [4]:
fullpath = os.path.join(mainpath,f1)
data = pd.read_csv(fullpath)

data['Hora_Retiro'] = data['Hora_Retiro'].str.slice(0, 8)
data['Hora_Arribo'] = data['Hora_Arribo'].str.slice(0, 8)

df1 = pd.DataFrame(data)
duracion = pd.to_datetime(df1.Hora_Arribo) - pd.to_datetime(df1.Hora_Retiro)

df1['duracion'] = duracion
df1 = df1.drop(['Genero_Usuario', 'Edad_Usuario', 'Hora_Retiro', 'Fecha_Arribo', 'Hora_Arribo'], axis=1)
df1.columns = ['ID_bici', 'est_salida', 'fecha', 'est_llegada', 'duracion']
df1['ID'] = df1.index
df1 = df1[['ID','ID_bici', 'fecha', 'duracion', 'est_salida', 'est_llegada']]
df1['duracion'] = df1['duracion'].astype(str).str[-18:-10]
df1.head()


,ID,ID_bici,fecha,duracion,est_salida,est_llegada
0,0,69,2010-02-16,00:03:05,85,85
1,1,11,2010-02-16,00:28:54,85,26
2,2,43,2010-02-16,00:28:01,85,13
3,3,826,2010-02-16,00:16:33,22,85
4,4,662,2010-02-16,00:58:02,27,74


# First insertion

In this block of code, the first data set is inserted into the table.

In [5]:
for index, row in df1.iterrows():
    cursor.execute("INSERT INTO Viajes (ID,ID_bici,fecha,duracion,est_salida,est_llegada) values(?,?,?,?,?,?)", row.ID, row.ID_bici, row.fecha, row.duracion, row.est_salida, row.est_llegada)

# Second tranformations

Here, the second file goes through the same transformations that the first one got, so that everything gets the same format.

In [6]:
fullpath = os.path.join(mainpath,f2)
data = pd.read_csv(fullpath)

data['Hora_Retiro'] = data['Hora_Retiro'].str.slice(0, 8)
data['Hora_Arribo'] = data['Hora_Arribo'].str.slice(0, 8)

df2 = pd.DataFrame(data)
duracion2 = pd.to_datetime(df2.Hora_Arribo) - pd.to_datetime(df2.Hora_Retiro)

df2['duracion'] = duracion2
df2 = df2.drop(['Genero_Usuario', 'Edad_Usuario', 'Hora_Retiro', 'Fecha_Arribo', 'Hora_Arribo'], axis=1)
df2.columns = ['ID_bici', 'est_salida', 'fecha', 'est_llegada', 'duracion']
df2['ID'] = df2.index
df2 = df2[['ID','ID_bici', 'fecha', 'duracion', 'est_salida', 'est_llegada']]
df2['duracion'] = df2['duracion'].astype(str).str[-18:-10]
df2['fecha'] = pd.to_datetime(df2['fecha'], format='%d/%m/%Y')
df2.head()

,ID,ID_bici,fecha,duracion,est_salida,est_llegada
0,0,10427.0,2021-02-01,00:07:14,390.0,315
1,1,11429.0,2021-02-01,00:06:07,254.0,272
2,2,9959.0,2021-02-01,00:11:01,476.0,24
3,3,4044.0,2021-02-01,00:14:38,256.0,147
4,4,9986.0,2021-02-01,00:28:24,450.0,105


# Step before update

Since the primary key of the table is the index of the dataframe, the new ID has to start from the last ID inserted. Therefore, this block queries the max ID and overwrites the ID column of the dataframe from that number up.

Also, two of the columns have float data type, we have to change that in order to have the same format as the destiny table.

Something else to consider is that the second file has NULL values, since those rows that have NULL values in any column, have all columns as NULL, the treatment given is to just drop those rows.

In [7]:
sql = "SELECT MAX(ID) FROM Viajes"
cursor.execute(sql)
last_ID = cursor.fetchall()

total_rows = df2.shape[0]

n_first_ID = last_ID[0][0] + 1
n_last_ID = n_first_ID + total_rows

print(n_first_ID, n_last_ID)

n_ID= list(range(n_first_ID, n_last_ID))

df2['n_ID'] = n_ID

df2 = df2.drop(['ID'], axis=1)
df2 = df2[['n_ID','ID_bici', 'fecha', 'duracion', 'est_salida', 'est_llegada']]
df2.columns = ['ID','ID_bici', 'fecha','duracion','est_salida', 'est_llegada']

df3 = df2.dropna(axis = 0, how ='any')
#display(df2.dtypes)


df3['est_salida'] = df3['est_salida'].astype(int)
df3['ID_bici'] = df3['ID_bici'].astype(int)
df3.head()

1057 290842


/home/ale/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ale/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ID,ID_bici,fecha,duracion,est_salida,est_llegada
0,1057,10427,2021-02-01,00:07:14,390,315
1,1058,11429,2021-02-01,00:06:07,254,272
2,1059,9959,2021-02-01,00:11:01,476,24
3,1060,4044,2021-02-01,00:14:38,256,147
4,1061,9986,2021-02-01,00:28:24,450,105


# Update

Here, we simply insert the new values in the table.

In [ ]:
for index, row in df3.iterrows():
    cursor.execute("INSERT INTO Viajes (ID,ID_bici,fecha,duracion,est_salida,est_llegada) values(?,?,?,?,?,?)", row.ID, row.ID_bici, row.fecha, row.duracion, row.est_salida, row.est_llegada)

# Select

Here we do a simple SELECT query to check our results.
Since they are a lot, we'll just do a count.

In [ ]:
sql = "SELECT count(*) FROM Viajes"
cursor.execute(sql)
result = cursor.fetchall()
for i in result:
    print(i)